# Исследование объявлений о продаже квартир

В распоряжении  имеется архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Цель исследования — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

#### Оглавление
1. [Открытие файла и изучение общей информации](#start)
2. [Предобработка данных](#2)
3. [Вычисление и добавление дополнительных параметров в таблицу](#3)
4. [Проведение исследовательского анализа данных](#4)
5. [Общий вывод](#5)

### 1. Открытие файла с данными и изучение общей информации.  <a id='start'></a>

In [1]:
import pandas as pd
from IPython.display import display
df=pd.read_csv('datasets/real_estate_data.csv', sep='\t')
display(df.head(10))
print(df.info())
print(df.shape)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,...,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,...,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,...,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,...,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,...,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,...,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,...,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,...,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,...,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

Таблица с исходными данными имеет 23699 строк, т.е. в таблице имеются данные о 23699 квартирах. Столбцы total_images (число фотографий квартиры в объявлении), last_price (цена на момент снятия с публикации), total_area (площадь квартиры в квадратных метрах (м²)), first_day_exposition (дата публикации), rooms (число комнат), studio (квартира-студия (булев тип)), open_plan (свободная планировка (булев тип)) заполнены полностью, в остальных столбцах встречается различное количество пропусков.

In [2]:
print(df['total_images'].value_counts())


10    1798
9     1725
20    1694
8     1585
7     1521
6     1482
11    1362
5     1301
12    1225
0     1059
13    1015
14     986
4      986
15     948
1      872
3      769
16     761
17     650
18     642
2      640
19     603
23      16
21      12
24       8
22       8
26       5
28       4
32       4
50       3
29       3
31       2
35       2
30       2
27       2
37       1
39       1
25       1
42       1
Name: total_images, dtype: int64


In [3]:
print(df['last_price'].value_counts())
print(df['last_price'].describe())


4500000.0     342
3500000.0     291
4000000.0     260
4300000.0     260
4200000.0     259
             ... 
4412094.0       1
6416000.0       1
4581000.0       1
34551000.0      1
6029433.0       1
Name: last_price, Length: 2978, dtype: int64
count    2.369900e+04
mean     6.541549e+06
std      1.088701e+07
min      1.219000e+04
25%      3.400000e+06
50%      4.650000e+06
75%      6.800000e+06
max      7.630000e+08
Name: last_price, dtype: float64


In [4]:
print(df['total_area'].value_counts())
print(df['total_area'].describe())

45.00     419
42.00     383
60.00     347
31.00     346
44.00     345
         ... 
45.85       1
44.18       1
59.83       1
127.20      1
23.55       1
Name: total_area, Length: 2182, dtype: int64
count    23699.000000
mean        60.348651
std         35.654083
min         12.000000
25%         40.000000
50%         52.000000
75%         69.900000
max        900.000000
Name: total_area, dtype: float64


In [5]:
print(df['first_day_exposition'].value_counts())

2018-02-01T00:00:00    368
2017-11-10T00:00:00    240
2017-10-13T00:00:00    124
2017-09-27T00:00:00    111
2018-03-26T00:00:00     97
                      ... 
2016-09-25T00:00:00      1
2016-08-28T00:00:00      1
2016-09-05T00:00:00      1
2015-07-26T00:00:00      1
2018-05-20T00:00:00      1
Name: first_day_exposition, Length: 1491, dtype: int64


 Столбец first_day_exposition (дата публикации) не имеет пропущенных значений, но формат даты необходимо перевести в datetime.

In [6]:
print(df['rooms'].value_counts())

1     8047
2     7940
3     5814
4     1180
5      326
0      197
6      105
7       59
8       12
9        8
10       3
14       2
11       2
19       1
12       1
16       1
15       1
Name: rooms, dtype: int64


In [7]:
print(df['ceiling_height'].value_counts())
print(df['ceiling_height'].describe())


2.50      3515
2.60      1646
2.70      1574
3.00      1112
2.80       993
          ... 
27.50        1
5.00         1
3.59         1
4.80         1
100.00       1
Name: ceiling_height, Length: 183, dtype: int64
count    14504.000000
mean         2.771499
std          1.261056
min          1.000000
25%          2.520000
50%          2.650000
75%          2.800000
max        100.000000
Name: ceiling_height, dtype: float64


В столбце ceiling_height (высота потолков (м)) 14504 значения заполнены из 23699, максимальное значение столбца 100 м, что кажется сомнительным. Эти значения требуют более тщательного рассмотрения и дополнительного уточнения исходных данных, т.к. в типовых квартирах высота потолков не больше 4 м.

In [8]:
print(df['floors_total'].value_counts())

5.0     5788
9.0     3761
16.0    1376
12.0    1362
4.0     1200
10.0    1174
25.0    1075
6.0      914
17.0     833
3.0      668
7.0      592
14.0     553
18.0     505
24.0     469
8.0      390
2.0      383
15.0     365
23.0     352
19.0     339
22.0     286
20.0     271
13.0     229
11.0     203
27.0     164
21.0     158
26.0     124
1.0       25
35.0      24
28.0      21
36.0       3
29.0       1
60.0       1
33.0       1
52.0       1
37.0       1
34.0       1
Name: floors_total, dtype: int64


In [9]:
print(df['living_area'].value_counts())
print(df['living_area'].describe())

18.00    882
17.00    675
30.00    598
16.00    486
20.00    481
        ... 
20.03      1
10.55      1
24.83      1
20.97      1
15.55      1
Name: living_area, Length: 1782, dtype: int64
count    21796.000000
mean        34.457852
std         22.030445
min          2.000000
25%         18.600000
50%         30.000000
75%         42.300000
max        409.700000
Name: living_area, dtype: float64


В столбце living_area (жилая площадь в квадратных метрах(м²)) минимальное значение составляет 2м², что вызывает сомнения и требует более тщательного рассмотрения и дополнительного уточнения исходных данных.

In [10]:
print(df['floor'].value_counts())

2     3368
3     3073
1     2917
4     2804
5     2621
6     1305
7     1218
8     1083
9     1051
10     687
12     526
11     523
13     379
15     343
14     337
16     315
17     227
18     178
19     147
21     125
22     113
20     110
23     100
24      63
25      46
26      24
27      10
28       1
33       1
29       1
30       1
32       1
31       1
Name: floor, dtype: int64


In [11]:
print(df['is_apartment'].value_counts())

False    2725
True       50
Name: is_apartment, dtype: int64


Столбец "апартаменты" (is_apartment) имеет булев тип, его заполняли в случае, если ответ положительный (True) и, иногда, если отрицательный (False). Исходя из этих соображений пропущенный значения столбца is_apartment (апартаменты (булев тип)) заполним значениями False.

In [12]:
print(df['studio'].value_counts())

False    23550
True       149
Name: studio, dtype: int64


In [13]:
print(df['open_plan'].value_counts())

False    23632
True        67
Name: open_plan, dtype: int64


In [14]:
print(df['kitchen_area'].value_counts())
print(df['kitchen_area'].describe())

6.00     1300
10.00    1262
8.00     1110
9.00     1101
7.00     1062
         ... 
18.07       1
28.20       1
37.90       1
9.51        1
35.40       1
Name: kitchen_area, Length: 971, dtype: int64
count    21421.000000
mean        10.569807
std          5.905438
min          1.300000
25%          7.000000
50%          9.100000
75%         12.000000
max        112.000000
Name: kitchen_area, dtype: float64


In [15]:
print(df['balcony'].value_counts())

1.0    4195
0.0    3758
2.0    3659
5.0     304
4.0     183
3.0      81
Name: balcony, dtype: int64


В столбце "число балконов" не указывалось количество балконов, скорее всего, при их отсутствии, поэтому будет логично заменить пропуски на 0, а тип столбца следует изменить с float на int, т.к. количество балконов может быть только целым числом.

In [16]:
print(df['locality_name'].value_counts())

Санкт-Петербург                                            15721
посёлок Мурино                                               522
посёлок Шушары                                               440
Всеволожск                                                   398
Пушкин                                                       369
                                                           ...  
поселок Возрождение                                            1
садоводческое некоммерческое товарищество Лесная Поляна        1
деревня Большой Сабск                                          1
деревня Раздолье                                               1
поселок Калитино                                               1
Name: locality_name, Length: 364, dtype: int64


In [17]:
print(df['airports_nearest'].value_counts())

37434.0    61
21928.0    32
39946.0    30
44870.0    30
37407.0    27
           ..
25122.0     1
26768.0     1
22248.0     1
20475.0     1
14579.0     1
Name: airports_nearest, Length: 8275, dtype: int64


In [18]:
print(df['cityCenters_nearest'].value_counts())

8460.0     61
20802.0    32
10720.0    30
8434.0     27
20444.0    27
           ..
13104.0     1
11280.0     1
27218.0     1
10501.0     1
208.0       1
Name: cityCenters_nearest, Length: 7642, dtype: int64


In [19]:
print(df['parks_around3000'].value_counts())

0.0    10106
1.0     5681
2.0     1747
3.0      647
Name: parks_around3000, dtype: int64


In [20]:
print(df['parks_nearest'].value_counts())

441.0     67
173.0     41
392.0     41
456.0     40
471.0     32
          ..
863.0      1
2880.0     1
969.0      1
1011.0     1
775.0      1
Name: parks_nearest, Length: 995, dtype: int64


In [21]:
print(df['ponds_around3000'].value_counts())

0.0    9071
1.0    5717
2.0    1892
3.0    1501
Name: ponds_around3000, dtype: int64


In [22]:
print(df['ponds_nearest'].value_counts())

427.0     70
454.0     41
153.0     40
433.0     39
564.0     37
          ..
742.0      1
1019.0     1
948.0      1
1275.0     1
193.0      1
Name: ponds_nearest, Length: 1096, dtype: int64


In [23]:
print(df['days_exposition'].value_counts())
print(df['days_exposition'].describe())

45.0      880
60.0      538
7.0       234
30.0      208
90.0      204
         ... 
1110.0      1
1049.0      1
951.0       1
693.0       1
773.0       1
Name: days_exposition, Length: 1141, dtype: int64
count    20518.000000
mean       180.888634
std        219.727988
min          1.000000
25%         45.000000
50%         95.000000
75%        232.000000
max       1580.000000
Name: days_exposition, dtype: float64


В столбце days_exposition (сколько дней было размещено объявление (от публикации до снятия)) имеется 20518 заполненных значения из 23699 строк, причем, минимальное значение в этом столбце 1 день. Логично преположить, что в случае, если объявление еще активно, то в этой строке ничего не указывалось, т.е. можно заполнить пропуски значением 0, а так же поменять тип данных этого столбца с float на int, т.к. количество дней может быть только целым числом.

### Вывод

Таблица с исходными данными имеет 23699 строк, т.е. в таблице имеются данные о 23699 квартирах. Столбцы total_images (число фотографий квартиры в объявлении), last_price (цена на момент снятия с публикации), total_area (площадь квартиры в квадратных метрах (м²)), first_day_exposition (дата публикации), rooms (число комнат), studio (квартира-студия (булев тип)), open_plan (свободная планировка (булев тип)) заполнены полностью, в остальных столбцах встречается различное количество пропусков. В столбце ceiling_height (высота потолков (м)) 14504 значения заполнены из 23699, максимальное значение столбца 100 м, что кажется сомнительным. Эти значения требуют более тщательного рассмотрения и дополнительного уточнения исходных данных, т.к. в типовых квартирах высота потолков не больше 4 м. В столбце living_area (жилая площадь в квадратных метрах(м²)) минимальное значение составляет 2м², что вызывает сомнения и требует более тщательного рассмотрения и дополнительного уточнения исходных данных. 
В столбцах: airports_nearest (расстояние до ближайшего аэропорта в метрах (м)), balcony (число балконов), ceiling_height (высота потолков (м)), cityCenters_nearest (расстояние до центра города (м)), days_exposition (сколько дней было размещено объявление (от публикации до снятия)), floor (этаж), floors_total (всего этажей в доме), is_apartment (апартаменты (булев тип)), kitchen_area (площадь кухни в квадратных метрах (м²)), living_area (жилая площадь в квадратных метрах(м²)), locality_name (название населённого пункта), parks_around3000 (число парков в радиусе 3 км), parks_nearest (расстояние до ближайшего парка (м)), ponds_around3000 (число водоёмов в радиусе 3 км), ponds_nearest (расстояние до ближайшего водоёма (м)) имеется некоторое количество пропусков. 
В столбце "число балконов" не указывалось количество балконов, скорее всего, при их отсутствии, поэтому будет логично заменить пропуски на 0, а тип столбца следует изменить с float на int, т.к. количество балконов может быть только целым числом. Столбец "апартаменты" (is_apartment) имеет булев тип, его заполняли в случае, если ответ положительный (True) и, иногда, если отрицательный (False). Исходя из этих соображений пропущенный значения столбца is_apartment (апартаменты (булев тип)) заполним значениями False. В столбце days_exposition (сколько дней было размещено объявление (от публикации до снятия)) имеется 20518 заполненных значения из 23699 строк, причем, минимальное значение в этом столбце 1 день. Логично преположить, что в случае, если объявление еще активно, то в этой строке ничего не указывалось, т.е. можно заполнить пропуски значением 0, а так же поменять тип данных этого столбца с float на int, т.к. количество дней может быть только целым числом.
Столбец first_day_exposition (дата публикации) не имеет пропущенных значений, но формат даты необходимо перевести в datetime.

### 2. Предобработка данных<a id='2'></a>

In [24]:
#обработка пропущенных значений
df['balcony']=df['balcony'].fillna(0)
df['is_apartment']=df['is_apartment'].fillna(False)
df['days_exposition']=df['days_exposition'].fillna(0)
print(df['is_apartment'].value_counts())

False    23649
True        50
Name: is_apartment, dtype: int64


В столбцах: balcony (число балконов), days_exposition (сколько дней было размещено объявление (от публикации до снятия)), is_apartment (апартаменты (булев тип)) были заменены пропущенные значения. 

In [25]:
#обработка дубликатов
print(df.duplicated().sum())
print(df['locality_name'].duplicated().sum())

0
23334


Анализ методом duplicated().sum() дубликатов не обнаружил.

In [26]:
#замена типов данных
df['balcony']=df['balcony'].astype('int')
df['locality_name']=df['locality_name'].astype('str')
df['days_exposition']=df['days_exposition'].astype('int')
df['first_day_exposition']=pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   total_images          23699 non-null  int64         
 1   last_price            23699 non-null  float64       
 2   total_area            23699 non-null  float64       
 3   first_day_exposition  23699 non-null  datetime64[ns]
 4   rooms                 23699 non-null  int64         
 5   ceiling_height        14504 non-null  float64       
 6   floors_total          23613 non-null  float64       
 7   living_area           21796 non-null  float64       
 8   floor                 23699 non-null  int64         
 9   is_apartment          23699 non-null  bool          
 10  studio                23699 non-null  bool          
 11  open_plan             23699 non-null  bool          
 12  kitchen_area          21421 non-null  float64       
 13  balcony         

Были изменены типы данных в столбцах balcony (число балконов) и days_exposition (сколько дней было размещено объявление (от публикации до снятия)), locality_name(название населённого пункта) и first_day_exposition (дата публикации).

In [27]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
lemmas=[]
for i in range(df.shape[0]):
    lemmas += m.lemmatize(df['locality_name'][i])
print(Counter(lemmas).most_common(200))

df['locality_name']=df['locality_name'].str.replace('ё', 'е')
print(len(df['locality_name'].unique()))
print(df['locality_name'].unique())

ModuleNotFoundError: No module named 'pymystem3'

В некоторых случаях название населённого пункта было написано как "посёлок...", а в остальных-как "поселок...". Следует привести к одинаковому написанию, чтобы не было дублирования названия одного и того же населённого пункта, как, например, 'посёлок Пансионат Зелёный Бор' и 'поселок Пансионат Зелёный Бор'. Из этих соображений буква ё была заменена на е во всём столбце.

In [ ]:
#проверка наличия дубликатов после внесения изменений в столбец locality_name
print(df.duplicated().sum())
print(df['locality_name'].duplicated().sum())

После внесения изменений в столбец locality_name (название населённого пункта) новые дубликаты не появились

### Вывод

Были изменены типы данных в столбцах balcony (число балконов) и days_exposition (сколько дней было размещено объявление (от публикации до снятия)), locality_name(название населённого пункта) и first_day_exposition (дата публикации). Анализ методом duplicated().sum() дубликатов не обнаружил. В столбцах: balcony (число балконов), days_exposition (сколько дней было размещено объявление (от публикации до снятия)), is_apartment (апартаменты (булев тип)) были заменены пропущенные значения. Был проведен анализ уникальных значений и лемматизация столбца locality_name (название населённого пункта). В некоторых случаях название населённого пункта было написано как "посёлок...", а в остальных-как "поселок...". Следует привести к одинаковому написанию, чтобы не было дублирования названия одного и того же населённого пункта, как, например, 'посёлок Пансионат Зелёный Бор' и 'поселок Пансионат Зелёный Бор'. Из этих соображений буква ё была заменена на е во всём столбце. 

### 3. Вычисление и добавление дополнительных параметров в таблицу <a id='3'></a>

In [ ]:
#добавляем в таблицу столбец с ценой квадратного метра
df['price_kvm']=df['last_price']/df['total_area']
#добавляем в таблицу столбец с днем недели публикации объявления, где 0-понедельник, а 6- воскресенье
df['weekday']=df['first_day_exposition'].dt.weekday
#добавляем в таблицу столбец с номером месяца публикации объявления
df['month']=df['first_day_exposition'].dt.month
#добавляем в таблицу столбец с годом публикации объявления
df['year']=df['first_day_exposition'].dt.year
#добавляем в таблицу столбец с категорией этажа(первый, последний и другое)
def floor_key(row):
    floor=row['floor']
    floors_total=row['floors_total']
    if floor==1:
        return 'первый'
    if floor==floors_total:
        return 'последний'
    else:
        return 'другое'
df['floor_name']=df.apply(floor_key, axis=1)
#добавляем в таблицу соотношение жилой площади к общей площади квартиры
df['living/total']=df['living_area']/df['total_area']
#добавляем в таблицу соотношение площади кухни к общей площади квартиры
df['kitchen/total']=df['kitchen_area']/df['total_area']
print(df.head())

### Вывод

В исходную таблицу были добавлены столбцы: с ценой квадратного метра 'price_kvm', равного отношению цены('last_price') к общей площади квартиры('total_area'); с порядковым номером дня недели публикации объявления, где 0-понедельник, а 6- воскресенье; с номером месяца публикации объявления; с годом публикации объявления; с категорией этажа(первый, последний и другое); с соотношением жилой площади к общей площади квартиры (df['living_area']/df['total_area']); с соотношением площади кухни к общей площади квартиры (df['kitchen_area']/df['total_area']).

### 4. Проведение исследовательского анализа данных <a id='4'></a>

Анализ параметров: площадь, цена, число комнат, высота потолков

In [ ]:
import matplotlib.pyplot as plt
df.hist('total_area', bins=100,range=(0,100))

print(df['total_area'].value_counts().head(20))
print(df['total_area'].describe())

Наибольше количество объявлений было о продаже квартир площадью от 31 до 45 м² и около 60 м² и 80 м². 

In [ ]:
df.hist('rooms', bins=50,range=(0,20))
print(df['rooms'].value_counts())
print(df['rooms'].describe())

Наибольше количество объявлений было о продаже 1-3 комнатных квартир. 

In [ ]:
df.hist('ceiling_height', bins=100,range=(0,20))
print(df['ceiling_height'].value_counts())
print(df['ceiling_height'].describe())

Высота потолков (столбец ceiling_height), судя по соответствующей гистограмме, в основном, стандартная от 2,5 до 3,5 м, но есть и выпадающие значения: 25 и 100м.

In [ ]:
df.hist('last_price', bins=100)
df.hist('last_price', bins=100, range=(0,20000000))
plt.title('Ограничение цены: от 0 до 20000000')
print(df['last_price'].value_counts())
print(df['last_price'].describe())

Анализ столбца last_price (цена на момент снятия с публикации) показал, что, в основном, квартиры стоят 3-5млн, а наиболее частой ценой является - 4500000р.

### Вывод

Анализ столбцов rooms (количество комнат) и total_area(общая площадь) показал, что наибольше количество объявлений было о продаже 1-3 комнатных типовых квартир площадью от 31 до 45 м² и около 60 м² и 80 м². В столбце rooms (количество комнат) 197 строк имеет значение 0, возможно, это ошибка ввода или речь шла о продаже комнат, а не полноценных квартир. Объявления о продаже квартир, имеющих больше 5 комнат, встречаются сравнительно редко. Высота потолков (столбец ceiling_height), судя по соответствующей гистограмме, в основном, стандартная от 2,5 до 3,5 м, но есть и выпадающие значения: 25 и 100м.

#### Изучение времени продажи квартиры. Построение гистограммы. Вычисление среднего и медианы. 

In [ ]:
mean_days=df['days_exposition'].mean()
median_days=df['days_exposition'].median()
print('Среднее время продажи квартир: ', format(mean_days))
print('Медиана времени продажи квартир: ', format(median_days))
print(df['days_exposition'].describe())
print(df['days_exposition'].value_counts())
df.hist('days_exposition', bins=100)
df1=df.query('days_exposition>0')
mean_days1=df1['days_exposition'].mean()
median_days1=df1['days_exposition'].median()
print('Среднее время продажи квартир: ', format(mean_days))
print('Медиана времени продажи квартир: ', format(median_days))
print(df1['days_exposition'].describe())
print(df1['days_exposition'].value_counts())
df1.hist('days_exposition', bins=100)

Увеличим последнюю получившуюся гистограмму, чтобы рассмотреть подробнее возможный всплекс

In [ ]:
df1.hist('days_exposition', bins=40, range=(0,300))
df1.hist('days_exposition', bins=100)

На увеличенной гистограмме хорошо видно, что встречаются всплески примерно около 50, 75 и 90го дня

### Вывод

Среднее время продажи квартир около 156 дней, медиана времени продажи квартир - 74 дня. Наиболее часто встречается значение 0, что значит, что на момент сбора данных в таблицу объявление было активно, т.е. квартира ещё не продана. Для анализа скорости продажи квартир подходят только записи об уже проданных квартирах, поэтому при дальнейшем анализе строки столбца days_exposition, содержащие значения=0, не будут учтены. В таблицу df1 были собраны данные данные для которых значения столбца days_exposition>0. Медиана и среднее значение не изменились, что значит, что чаще всего(что хорошо видно на гистограмме) продажа квартиры занимает около 74 дней. Можно сказать, что квартира продалась быстро, если поиск покупателей занял 45 дней и меньше, и квартира продалась медленно, если на это потребовалось более 232 дней.

#### Исключение редких и выбивающихся значений и выявление особенностей

In [ ]:
df2=df1.query('days_exposition<400')
a=len(df2)/len(df)
b=len(df2)/len(df1)
print('При выделении значений 0<days_exposition<400 оставим {:.0%} от начальной таблицы или {:.0%} от таблицы, расмотренной в прошлом пункте, содержащей информацию только о завершенных сделках(т.е. days_exposition>0)'.format(a, b))
df2.hist('days_exposition', bins=100)
print(df2['days_exposition'].describe())
print(df2['days_exposition'].value_counts())

### Вывод

На значении 400 дней можно обрезать гистограмму, т.к. чем больше количество дней, тем меньше частота таких объявлений. Сохраним эту часть таблицы в df2. На гистограмме, построенной по новой таблице df2, т.е. содержащей записи об объявлениях с условием 0<days_exposition<400, ярко видны всплески: 7 дней, 45, 60 и 90 дней. При выделении значений 0<days_exposition<400 оставим 75% от начальной таблицы или 87% от таблицы, расмотренной в прошлом пункте, содержащей информацию только о завершенных сделках(т.е. days_exposition>0)

#### Выявление факторов, влияющих на стоимость квартиры. Зависимость цены от квадратного метра, числа комнат, этажа (первого или последнего), удалённости от центра, от даты размещения: дня недели, месяца и года. Выявление 10 населённых пунктов с наибольшим числом объявлений. Вычисление средней цену квадратного метра в этих населённых пунктах. Выявление населённых пунктов с самой высокой и низкой стоимостью жилья. 

In [ ]:
import matplotlib.pyplot as plt
df3=df[['last_price', 'total_area', 'rooms', 'floor_name', 'cityCenters_nearest']]
df4=df3[['last_price', 'total_area', 'rooms', 'cityCenters_nearest']]
pd.plotting.scatter_matrix(df3, figsize=(12, 12))
print('Коэффициент корреляции между общей площадью квартиры и ценой:', df3['last_price'].corr(df3['total_area']))
print('Коэффициент корреляции между количеством комнат и ценой:', df3['last_price'].corr(df3['rooms']))
print('Коэффициент корреляции между расстоянием до центра и ценой:', df3['last_price'].corr(df3['cityCenters_nearest']))

In [ ]:
for name, group_data in df3.groupby('floor_name'):
    group_data.plot(y='last_price', title=name)
table_floor=df3.pivot_table(index='floor_name', values='last_price', aggfunc='median')
table_floor.columns=['median_price']
table_floor.plot()
display(table_floor)

In [ ]:
location=df.pivot_table(index='locality_name', values='last_price', aggfunc=['count'])
location.columns=['amount']
top_location=location.sort_values(by='amount', ascending=False).head(10)
top_location_price=df.query('locality_name in @top_location.index')
print('10 населённых пунктов с наибольшим количеством объявлений: ', top_location.index)
top_location_price_table=top_location_price.pivot_table(index='locality_name', values='price_kvm', aggfunc='mean')
top_location_price_table.columns=['mean_price_for_kvm']
display(top_location_price_table)
print(top_location)
x1=top_location_price_table.sort_values(by='mean_price_for_kvm', ascending=False)
print('Наибольшая стоимость жилья: ', x1.head(1))
x2=top_location_price_table.sort_values(by='mean_price_for_kvm', ascending=True)
print('Наименьшая стоимость жилья: ', x2.head(1))

In [ ]:
list_col=['weekday', 'month', 'year']

for i in list_col:
    table=df.pivot_table(index=i, values='last_price', aggfunc='median')
    table.columns=['median_price']
    table.plot()
    display(table)

### Вывод

Из матрицы корреляции можно сделать вывод, что цена линейно зависит от общей площади квартиры, числа комнат, а так же от расположения: чем ближе к центру, тем выше цена, ближе всех к 1 коэффициент корреляции между общей площадью и ценой: 0.65, т.е. влияние общей площади на цену квартиры сильнее других параметров. Стоит отметить, что цена зависит от близости к центру до отметки в 20000м, далее цена от близости к центру не зависит. Была исследована зависимость цены от даты размещения: дня недели, месяца и года. На графиках можно видеть, что медианное значение цены наибольшее для объявлений, размещенных во вторник, в апреле и в 2014году. Цена квартир линейно падает в зависимости от дня недели размещения от максимального значения во вторник до минимального значения в субботу и воскресенье. Так же наблюдается минимум цены квартиры для объявлений, размещенных в июне. Возможно, такая особенность связана с сезоном отпусков, т.е. люди летом продают недвижимость только в случае острой необходимости и готовы продавать значительно дешевле. Так же наблюдается падение цен на квартиры от 2014 года(максимальное значение) до 2017 (минимальное значение), а затем, в 2018-2019 - небольшой плавный рост. В рамках анализа были выбраны населнные пункты с наибольшим количеством объявлений. Ими стали: 'Санкт-Петербург', 'поселок Мурино', 'поселок Шушары', 'Всеволожск',        'Пушкин', 'Колпино', 'поселок Парголово', 'Гатчина', 'деревня Кудрово', 'Выборг'; из них наибольшая стоимость квартиры в Санкт-Петербурге - медианное значение: 114849.008794, а наименьшая - в Выборге - медианное значение: 58141.909153.

#### График зависимости цены от удалённости от центра

In [ ]:
df6=df
#найдем и уберем пустые значения в столбце, описывающем расстояние до центра
df6['cityCenters_nearest'].isnull().sum()
df6.dropna(subset=['cityCenters_nearest'], inplace=True)
df6['cityCenters_nearest_km']=df6['cityCenters_nearest']/1000
df6['cityCenters_nearest_km']=df6['cityCenters_nearest_km'].astype('int')
#выделим только квартиры в Санкт-Петербурге
df7=df6.query('locality_name=="Санкт-Петербург"')
print(df7['cityCenters_nearest_km'].value_counts())
#print(df7['cityCenters_nearest_km'].describe())
#df6=df7[['locality_name', 'cityCenters_nearest_km', 'last_price']]
table_distance=df7.pivot_table(index='cityCenters_nearest_km', values='last_price', aggfunc='mean')
table_distance.columns=['mean_last_price']
display(table_distance)
table_distance.plot(grid=True, xlim=(0, 27))


### Вывод

Анализ количества объявлений о продаже квартир в зависимости от расстояния до центра Санкт-Петербурга показывает, что наибольшее количество объявлений было о квартирах, находящихся на расстоянии 10-15км от центра, а о квартирах на расстоянии меньше 10 км - несколько меньше, но стоит отметить, что наименьшее количество объявлений о продажах квартир на максимальном расстоянии, т.е. 20 км и более. Рассмотрим график зависимости среднего значения цены квартиры от расстояния до центра: от 5км до 8км от центра наблюдается резкое падение среднего значения цены и выход линии графика на планку. Отсюда можно сделать вывод, что центральной зоной можно считать районы, расположеные на расстоянии до 8км.

#### Анализ сегмента квартир в центре

In [ ]:
#выделим сегмент квартир в центре
df8=df7.query('cityCenters_nearest_km<=8')
#df8.hist('total_area', bins=100, range=(0, 200))
print(df8['total_area'].value_counts())
print(df8['rooms'].value_counts())
print(df8['ceiling_height'].value_counts())
print(df8.sort_values(by='last_price', ascending=False).head(10))
df8_price=df8.pivot_table(index='cityCenters_nearest_km', values='last_price', aggfunc='median')
df8_price.columns=['median_last_price']
display(df8_price)
df8.plot(x='ceiling_height', y='last_price', xlim=(2,4), style='o')
df8_rooms=df8.pivot_table(index='rooms', values='last_price', aggfunc='median')
#df8_rooms.hist(bins=10, range=(0, 10))
df8_rooms.columns=['median_last_price']
display(df8_rooms)
df8_height=df8.pivot_table(index='ceiling_height', values='last_price', aggfunc='median')
#df8_height.hist(bins=50, range=(2, 10))
df8_height.columns=['median_last_price']
display(df8_height)
df9=df8[['total_area', 'rooms','ceiling_height','last_price']]
df10=df7[['total_area', 'rooms','ceiling_height','last_price']]
pd.plotting.scatter_matrix(df9, figsize=(12, 12))
plt.title('в центре')
pd.plotting.scatter_matrix(df10, figsize=(12, 12))
plt.title('по всему городу')
df8.pivot_table(index='floor', values='last_price', aggfunc='median').plot()
df8.pivot_table(index='cityCenters_nearest_km', values='last_price', aggfunc='median').plot()
df8.pivot_table(index='year', values='last_price', aggfunc='median').plot()
df8.pivot_table(index='month', values='last_price', aggfunc='median').plot()
df8.pivot_table(index='weekday', values='last_price', aggfunc='median').plot()
df8.pivot_table(index='weekday', values='price_kvm', aggfunc='median').plot()
df8.pivot_table(index='first_day_exposition', values='last_price', aggfunc='median').plot()
df7.pivot_table(index='floor', values='last_price', aggfunc='median').plot(title='по всему городу')
df7.pivot_table(index='cityCenters_nearest_km', values='last_price', aggfunc='median').plot(title='по всему городу')
df7.pivot_table(index='year', values='last_price', aggfunc='median').plot(title='по всему городу')
df7.pivot_table(index='month', values='last_price', aggfunc='median').plot(title='по всему городу')
df7.pivot_table(index='weekday', values='last_price', aggfunc='median').plot(title='по всему городу')
df7.pivot_table(index='weekday', values='price_kvm', aggfunc='median').plot(title='по всему городу')
df7.pivot_table(index='first_day_exposition', values='last_price', aggfunc='median').plot(title='по всему городу')

### Вывод

Анализ влияния параметров: площадь, цена, число комнат, высота потолков показал, что наиболее часто в центре выставляют на продажу 3 и 2-х комнатные квартиры типой застройки с высотой потолков 3м. Так же можно заметить из матрицы корреляций, что цена линейно зависит от площади, но от количества комнат цена зависит не так явно. Разгадка может быть в том, что при одной и той же площади в разных типах квартир - разное количество комнат, поэтому цена более явно привязана к площади квартиры, а количество комнат и высота потолков больше характеризуют тип дома. Из графиков, описывающих зависимость цены от даты публикации, этажа и расстояния от центра, можно сделать вывод, что цена имеет максимум при этаже, равном 25(это максимальное значение этажа в центральном районе); имеется некоторое падение цены при удаленности от центра на 3км и резкий рост - при 4 км, но при дальнейшем увеличении расстояния цена продолжает падать; цены на квартиры упали до минимума в 2017-2018 годах, а своего максимума достигали в 2014г.; цена идет на резкий спад только к январе, а в остальных месяцах немного увеличивает в нечтные месяцы, а в чётные-можно заметить некоторое падение; в субботу и во вторник самые низкие цены. Сравнение зависимостей медианных значений цены квартиры и стоимости квадратного метра квартиры в зависимости от дня недели говорит о том, что во вторник чаще выставляют на продажу наиболее дешёвые квартиры, а в субботу квартиры имеют минимальную цену, но при этом, цена за квадратный метр близка к максимуму, т.е. можно сделать вывод, что субботу выставляют на продажу чаще небольшие квартиры. Если посмотреть на графики тех же зависимостей, но построенные по данным по всему городу, то можно заметить, что цена начинает резко расти при величине этаже выше 27 и достигает максимума на отметке - 30, затем падает до приблизительно среднего значения на отметке этажа - 32 и снова резко растет вверх. График зависимости цены от расстояния от центра демонстрирует, что цена при растоянии выше 8км выходит на планку, но имеет небольшой скачок вверх на отметке 20км и достаточно существенный скачок на отметке 27-28км. График зависимости цены от года публикации практически такой же, как и аналогичный, построенный по данным квартир, расположенных в центральном районе, что говорит о падении цен на недвижимость в 2015-2017 годах. Зависимость цены от месяца публикации, построенная по данным квартир, расположеннных по всему городу, показывает, что цены минимальны в июне и октябре, а в декабре-в отличие от цен на квартиры в центре-достигает максимального значения. Стоит отметить, что зависимость цены от дня недели публикации объявления существенно отличается от объявлений о продаже квартир, находящихся в центре: цены на квартиры, находящиеся по всему городу максимальны в четверг и падают до минимума в воскресенье, а в остальные дни держатся ровно на одном уровне, хотя так же, как и на графике зависимости цены квадратного метра от дня недели-цена квадратного метра минимальна в субботу.

### 5. Общий вывод <a id='5'></a>

В результате проведенного анализа были сделаны выводы: 
**1** Анализ столбцов rooms (количество комнат) и total_area(общая площадь) показал, что наибольше количество объявлений было о продаже 1-3 комнатных типовых квартир площадью от 31 до 45 м² и около 60 м² и 80 м² с высотой потолков от 2,5 м до 3,5 м. 
**2** Среднее время продажи квартир около 156 дней, медиана времени продажи квартир - 74 дня. Можно сказать, что квартира продалась быстро, если поиск покупателей занял 45 дней и меньше, и квартира продалась медленно, если на это потребовалось более 232 дней. На графиках, приведенных выше, ярко видны всплески: 7 дней, 45, 60 и 90 дней размещения объявления до продажи квартиры.
**5** Цена квартиры сильно зависит от общей площади, и от расположения: чем ближе к центру, тем выше цена. В результате анализа цен был сделан вывод, что центральной зоной Санкт-Петербурга можно считать районы, расположеные на расстоянии до 8км. Анализ продажи квартир, расположенных в Санкт-Петербурге показал, что цена начинает резко расти при величине этаже выше 27 и достигает максимума на отметке - 30, затем падает до приблизительно среднего значения на отметке этажа - 32 и снова резко растет вверх. График зависимости цены от расстояния от центра демонстрирует, что цена при растоянии выше 8км выходит на планку, но имеет небольшой скачок вверх на отметке 20км и достаточно существенный скачок на отметке 27-28км. 
**6** Медианное значение цены наибольшее для объявлений, размещенных во вторник, в апреле и в 2014году. Цена квартир линейно падает в зависимости от дня недели размещения от максимального значения во вторник до минимального значения в субботу и воскресенье. Так же наблюдается минимум цены квартиры для объявлений, размещенных в июне. Так же наблюдается падение цен на квартиры от 2014 года(максимальное значение) до 2017 (минимальное значение), а затем, в 2018-2019 - небольшой плавный рост. 
**7*** Населённые пункты с наибольшим количеством объявлений: 'Санкт-Петербург', 'поселок Мурино', 'поселок Шушары', 'Всеволожск', 'Пушкин', 'Колпино', 'поселок Парголово', 'Гатчина', 'деревня Кудрово', 'Выборг'. Из них наибольшая стоимость квартиры в Санкт-Петербурге - медианное значение: 114849.008794, а наименьшая - в Выборге - медианное значение: 58141.909153.